<a href="https://colab.research.google.com/github/gabipalacios/Python/blob/main/ModeloZeros_VALOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
import datetime as dt
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sn
from google.colab import drive
pd.options.display.float_format = '{:.2f}'.format

drive.mount('/content/drive')
origen = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_sinpagos3.csv')
marco = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_PreMarcoBCU3.csv')

#Variables seleccionadas del maro:
marco = marco.add_prefix('m_')
#Variables seleccionadas de Financial y PLataforma
origen = origen.add_prefix('o_')

#Elimino campos,tratando de evitar riesgos de plataforma y campos repetidos y con inconvenientes, (analizados previamente)
#origen = origen.drop(['o_Solicitud','o_dias_atraso_max','o_TEA'], axis=1)

#join de las dos tablas
df = pd.merge(origen, marco, left_on='o_Documento', right_on='m_Documento', how='inner')
#Defino índice documento
df = df.set_index('m_Documento')
#Quito registros que no tienen score
#df = df.dropna(subset=['o_clr_Score'])

#Reorganizo valores de variables con criterio lógico, para obtener mejore conclusiones, (analizados previamente)
df.loc[~df['o_dispositivo_marca'].isin(['Apple']), 'o_dispositivo_marca'] = 'Android'
df.loc[df['o_sist_op_familia'] != 'iOS', 'o_sist_op_familia'] = 'Android'
df.loc[df['o_dispositivo_marca'] != 'Apple', 'o_dispositivo_marca'] = 'Android'
df.loc[~df['o_navegador_familia'].isin(['Facebook', 'Instagram']), 'o_navegador_familia'] = 'Web'
df.loc[~df['o_departamento'].isin(['MONTEVIDEO', 'CANELONES']), 'o_departamento'] = 'RESTO'
#df['o_Zero'] = np.where(df['o_Zero'] == 'Zero', 1, 0)
#df['o_clr_accion'] = np.where(df['o_clr_accion'].isin(['C2', 'C1_3_3','C4','C1_2_3','C1_3_6','C1_3_1','C3']), df['o_clr_accion'], 'Varios')

print(df.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1825, 53)


In [31]:
print(marco.shape)
print(origen.shape)
print(df.shape)

(2445, 20)
(2746, 34)
(1825, 53)


In [32]:
df = df.drop(['o_Documento'], axis=1)

In [33]:
marco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   m_Documento           2445 non-null   int64  
 1   m_Deuda               2445 non-null   int64  
 2   m_DeudaPrincipal      2445 non-null   int64  
 3   m_DeudaSec_Prom       2445 non-null   int64  
 4   m_Entidades           2445 non-null   int64  
 5   m_TieneBco            2445 non-null   int64  
 6   m_TieneFin            2445 non-null   int64  
 7   m_Rel_Deuda_Fin       2445 non-null   float64
 8   m_Rel_DeudaPrincipal  2445 non-null   float64
 9   m_Rel_Contingencia    2445 non-null   float64
 10  m_Var_dueda_6M        2445 non-null   float64
 11  m_Deuda_Var-6         2444 non-null   float64
 12  m_Rel_VarDeuda_Deuda  2419 non-null   float64
 13  m_1PEORCATBCU6M       2445 non-null   object 
 14  m_Sin_Ficha           2445 non-null   int64  
 15  m_Malos              

In [34]:
#Modificar el parámetro de la tabla df['hora'] donde se muestran las horas del día desde 0 a 23,
#son tipos de datos periódicos, utilizando el cálculo trigonométrico más adecuado para introducir el parámetro en un modelo predictivo.
df['o_Hora'] = pd.to_numeric(df['o_Hora'])

# Calcular la representación sinusoidal de la hora
df['hora_sin'] = np.sin(2 * np.pi * df['o_Hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['o_Hora'] / 24)

# Eliminar la columna original 'hora'
df = df.drop('o_Hora', axis=1)


In [35]:
origen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746 entries, 0 to 2745
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   o_Date                 2746 non-null   object 
 1   o_Documento            2746 non-null   int64  
 2   o_fpd4                 2746 non-null   int64  
 3   o_quincena             2746 non-null   int64  
 4   o_Hora                 2746 non-null   int64  
 5   o_Num_Dia_Sem          2746 non-null   int64  
 6   o_canal                2746 non-null   object 
 7   o_status_usuario       2746 non-null   object 
 8   o_status               2746 non-null   object 
 9   o_sist_op_familia      2738 non-null   object 
 10  o_cargo                2745 non-null   object 
 11  o_relacion_laboral     2745 non-null   object 
 12  o_nivel_educativo      2745 non-null   object 
 13  o_navegador_familia    2738 non-null   object 
 14  o_dispositivo_familia  2738 non-null   object 
 15  o_de

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 10994184 to 9531591
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   o_Date                 1825 non-null   object 
 1   o_fpd4                 1825 non-null   int64  
 2   o_quincena             1825 non-null   int64  
 3   o_Num_Dia_Sem          1825 non-null   int64  
 4   o_canal                1825 non-null   object 
 5   o_status_usuario       1825 non-null   object 
 6   o_status               1825 non-null   object 
 7   o_sist_op_familia      1825 non-null   object 
 8   o_cargo                1825 non-null   object 
 9   o_relacion_laboral     1825 non-null   object 
 10  o_nivel_educativo      1825 non-null   object 
 11  o_navegador_familia    1825 non-null   object 
 12  o_dispositivo_familia  1822 non-null   object 
 13  o_departamento         1825 non-null   object 
 14  o_localidad            1824 non-null   object 
 15 

In [37]:
print(df.o_relacion_laboral.unique())
df.o_relacion_laboral.value_counts()

['jubilado' 'indep_sin_local' 'emp_privado' 'indep_con_local' 'otros'
 'emp_publico']


,count
o_relacion_laboral,
emp_privado,668
indep_sin_local,339
jubilado,227
emp_publico,214
indep_con_local,192
otros,185


In [38]:
boolean_like_cols = []
for col in df.columns:
  if df[col].dtype == 'object':
    unique_values = df[col].unique()
    if len(unique_values) <= 2:
      boolean_like_cols.append(col)
  elif df[col].dtype in ['int64', 'float64']:
    if df[col].nunique() <= 2 and df[col].min() in [0, 1] and df[col].max() in [0, 1]:
       boolean_like_cols.append(col)

print("Columnas con aspecto de booleano:")
boolean_like_cols

Columnas con aspecto de booleano:


['o_fpd4',
 'o_canal',
 'o_status',
 'o_sist_op_familia',
 'o_genero',
 'o_dispositivo_marca',
 'o_clr_Cancelaciones',
 'o_clr_Incumplimientos',
 'o_cdr_tiene_quita',
 'o_cdr_tiene_oca',
 'o_cdr_tiene_brou',
 'm_TieneBco',
 'm_TieneFin']

In [39]:
# prompt: convertir las siguientes variables a boolean: [ 'o_cdr_tiene_quita', 'o_cdr_tiene_oca', 'o_cdr_tiene_financiera',
#                      'o_cdr_tiene_brou', 'o_cdr_tiene_banco_privado', 'm_Contingencia',
#                      'm_Var_dueda_3M']

cols_to_convert_to_boolean = ['o_fpd4',
 'o_canal',
 'o_status',
 'o_sist_op_familia',
 'o_genero',
 'o_dispositivo_marca',
 'o_clr_Cancelaciones',
 'o_clr_Incumplimientos',
 'o_cdr_tiene_quita',
 'o_cdr_tiene_oca',
 'o_cdr_tiene_brou',
 'm_TieneBco',
 'm_TieneFin']

for col in cols_to_convert_to_boolean:
    # Assuming the columns are currently object type with values like 'True', 'False', 1, 0, etc.
    # This conversion handles common string and numeric representations of boolean.
    df[col] = df[col].astype(bool)

df[cols_to_convert_to_boolean].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 10994184 to 9531591
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   o_fpd4                 1825 non-null   bool 
 1   o_canal                1825 non-null   bool 
 2   o_status               1825 non-null   bool 
 3   o_sist_op_familia      1825 non-null   bool 
 4   o_genero               1825 non-null   bool 
 5   o_dispositivo_marca    1825 non-null   bool 
 6   o_clr_Cancelaciones    1825 non-null   bool 
 7   o_clr_Incumplimientos  1825 non-null   bool 
 8   o_cdr_tiene_quita      1825 non-null   bool 
 9   o_cdr_tiene_oca        1825 non-null   bool 
 10  o_cdr_tiene_brou       1825 non-null   bool 
 11  m_TieneBco             1825 non-null   bool 
 12  m_TieneFin             1825 non-null   bool 
dtypes: bool(13)
memory usage: 37.4 KB


In [40]:
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)


boolean_features = ['o_fpd4',
 'o_canal',
 'o_status',
 'o_sist_op_familia',
 'o_genero',
 'o_dispositivo_marca',
 'o_clr_Cancelaciones',
 'o_clr_Incumplimientos',
 'o_cdr_tiene_quita',
 'o_cdr_tiene_oca',
 'o_cdr_tiene_brou',
 'm_TieneBco',
 'm_TieneFin']

target = "o_fpd4"

for col in boolean_features + [target]:

    df[col] = df[col].astype(bool)

X = df.drop("o_fpd4", axis=1)
y = df["o_fpd4"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y, shuffle=True)

X_train.head()

,o_Date,o_quincena,o_Num_Dia_Sem,o_canal,o_status_usuario,o_status,o_sist_op_familia,o_cargo,o_relacion_laboral,o_nivel_educativo,o_navegador_familia,o_dispositivo_familia,o_departamento,o_localidad,o_ingreso_estimado,o_ingreso_declarado,o_genero,o_edad,o_dispositivo_marca,o_clr_Score3,o_clr_Cancelaciones,o_clr_Consultas,o_clr_Incumplimientos,o_cel_carrier,o_cdr_tiene_quita,o_cdr_tiene_oca,o_cdr_tiene_brou,o_cdr_segmento,o_pricing,o_email,o_scoring,m_Deuda,m_DeudaPrincipal,m_DeudaSec_Prom,m_Entidades,m_TieneBco,m_TieneFin,m_Rel_Deuda_Fin,m_Rel_DeudaPrincipal,m_Rel_Contingencia,m_Var_dueda_6M,m_Deuda_Var-6,m_Rel_VarDeuda_Deuda,m_1PEORCATBCU6M,m_Sin_Ficha,m_Malos,m_Grupo_b,m_Grupo_i,m_Q_irregular_12m,m_Situacion-6CDR,hora_sin,hora_cos
m_Documento,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
39134533,2025-03-10 00:00:00,1,2,True,id_verified,True,True,Cocinero,indep_con_local,Secundaria - Bachillerato,Facebook,Samsung SM-G985F,RESTO,DOLORES,22823.00,70000.00,True,42,True,582,False,3,False,ANTEL,False,True,False,1,4,gmail.com,624.00,246362,147640,49361,3,True,True,0.40,0.60,0.02,4.22,199192.00,0.81,1C,0,0,B1,B1,0,Buena,0.71,0.71
13824388,2024-12-27 00:00:00,2,6,True,id_verified,True,True,BPS / AFAP,jubilado,Educación Terciaria,Web,K,CANELONES,SOLYMAR,28193.00,35000.00,True,64,True,858,False,0,False,CLARO,False,False,True,7,7,hotmail.com,602.00,838278,624039,71413,4,True,True,0.15,0.74,0.00,0.12,88394.00,0.11,1C,0,0,C,C,0,Buena,-0.97,0.26
33169780,2025-01-08 00:00:00,1,4,True,id_verified,True,True,Administrativo / Contable / Oficinista,emp_privado,Secundaria - Bachillerato,Facebook,Samsung SM-A135M,MONTEVIDEO,MONTEVIDEO,101051.00,21000.00,True,58,True,475,False,0,True,CLARO,False,False,False,4,5,gmail.com,650.00,9180499,8878233,75567,5,True,True,1.00,0.97,0.02,0.01,127323.00,0.01,3,0,3,RR,C,0,Peor,-0.71,-0.71
46551762,2025-01-29 00:00:00,2,4,True,id_verified,True,True,Vendedor / Auxiliar Comercio o Servicios,emp_privado,Secundaria - Bachillerato,Instagram,iPhone,RESTO,SARANDI DEL YI,18169.00,32000.00,True,31,True,853,False,1,False,ANTEL,False,False,True,4,8,gmail.com,543.00,6550,6550,0,1,True,False,0.00,1.00,0.20,10.00,5955.00,0.91,1C,1,0,A,A,0,Buena,-0.97,0.26
32808214,2024-11-21 00:00:00,2,5,True,id_verified,True,True,Administrativo / Contable / Oficinista,emp_privado,Secundaria - Bachillerato,Facebook,2310FPCA4G,CANELONES,LAS PIEDRAS,50738.00,35105.00,True,51,True,149,False,1,False,MOVISTAR,False,True,False,3,5,gmail.com,353.00,230030,72802,15723,11,True,True,0.39,0.32,0.39,0.29,52230.00,0.23,1C,0,0,C,C,0,Buena,-0.26,-0.97


In [41]:
# prompt: Separar las variables en función del formato y dentro de el mismo de su distribución, preperarlas para utilizar en en un modelo predictivo.

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import numpy as np

# Identify numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist() # Exclude boolean here
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Include boolean features separately as they don't need scaling or encoding
boolean_features_df = X_train.select_dtypes(include=['bool'])
boolean_features_test_df = X_test.select_dtypes(include=['bool'])


print("Numerical features:", numerical_features)
print("Categorical features:", categorical_features)
print("Boolean features:", boolean_features_df.columns.tolist())

# Separate numerical features by distribution (e.g., identifying skewed distributions for transformation)
# This is a basic approach; more sophisticated methods might involve visual inspection or statistical tests.
skewed_numerical_features = []
for col in numerical_features:
    if X_train[col].nunique() > 20: # Check skew only for features with a decent number of unique values
        skew = X_train[col].skew()
        if abs(skew) > 0.5: # Threshold for considering a distribution skewed
            skewed_numerical_features.append(col)

print("Skewed numerical features (candidates for transformation):", skewed_numerical_features)

# Prepare data for the predictive model

# 1. Handle numerical features: Scaling and Imputation
# Identify and handle infinite values before scaling
for col in numerical_features:
    if np.isinf(X_train[col]).any():
        print(f"Column '{col}' contains infinite values in training data.")
        X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan)
    if np.isinf(X_test[col]).any():
        print(f"Column '{col}' contains infinite values in test data.")
        X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)


# Impute missing values with the mean (after replacing infinities with NaN)
for col in numerical_features:
    if X_train[col].isnull().any():
        mean_val = X_train[col].mean()
        X_train[col] = X_train[col].fillna(mean_val)
        X_test[col] = X_test[col].fillna(mean_val)


# Use MinMaxScaler to scale numerical features to a range [0, 1].
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_features])
X_test_scaled = scaler.transform(X_test[numerical_features])

# Convert scaled arrays back to DataFrames with appropriate column names
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=numerical_features, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=numerical_features, index=X_test.index)


# 2. Handle categorical features: One-Hot Encoding
# Use OneHotEncoder to convert categorical features into numerical representations.
# Handle potential new categories in the test set with 'handle_unknown='ignore''.
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train[categorical_features])
X_test_encoded = encoder.transform(X_test[categorical_features])

# Convert encoded arrays back to DataFrames with appropriate column names
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_features), index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_features), index=X_test.index)

# 3. Combine processed features
# Concatenate scaled numerical features, one-hot encoded categorical features, and boolean features
X_train_processed = pd.concat([X_train_scaled_df, X_train_encoded_df, boolean_features_df], axis=1)
X_test_processed = pd.concat([X_test_scaled_df, X_test_encoded_df, boolean_features_test_df], axis=1)

# Display the first few rows of the processed training data
display(X_train_processed.head())

# Display information about the processed dataframes
print("\nProcessed training data info:")
X_train_processed.info()

print("\nProcessed test data info:")
X_test_processed.info()

Numerical features: ['o_quincena', 'o_Num_Dia_Sem', 'o_ingreso_estimado', 'o_ingreso_declarado', 'o_edad', 'o_clr_Score3', 'o_clr_Consultas', 'o_cdr_segmento', 'o_pricing', 'o_scoring', 'm_Deuda', 'm_DeudaPrincipal', 'm_DeudaSec_Prom', 'm_Entidades', 'm_Rel_Deuda_Fin', 'm_Rel_DeudaPrincipal', 'm_Rel_Contingencia', 'm_Var_dueda_6M', 'm_Deuda_Var-6', 'm_Rel_VarDeuda_Deuda', 'm_Sin_Ficha', 'm_Malos', 'm_Q_irregular_12m', 'hora_sin', 'hora_cos']
Categorical features: ['o_Date', 'o_status_usuario', 'o_cargo', 'o_relacion_laboral', 'o_nivel_educativo', 'o_navegador_familia', 'o_dispositivo_familia', 'o_departamento', 'o_localidad', 'o_cel_carrier', 'o_email', 'm_1PEORCATBCU6M', 'm_Grupo_b', 'm_Grupo_i', 'm_Situacion-6CDR']
Boolean features: ['o_canal', 'o_status', 'o_sist_op_familia', 'o_genero', 'o_dispositivo_marca', 'o_clr_Cancelaciones', 'o_clr_Incumplimientos', 'o_cdr_tiene_quita', 'o_cdr_tiene_oca', 'o_cdr_tiene_brou', 'm_TieneBco', 'm_TieneFin']
Skewed numerical features (candidates f

/usr/local/lib/python3.11/dist-packages/pandas/core/nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean


,o_quincena,o_Num_Dia_Sem,o_ingreso_estimado,o_ingreso_declarado,o_edad,o_clr_Score3,o_clr_Consultas,o_cdr_segmento,o_pricing,o_scoring,m_Deuda,m_DeudaPrincipal,m_DeudaSec_Prom,m_Entidades,m_Rel_Deuda_Fin,m_Rel_DeudaPrincipal,m_Rel_Contingencia,m_Var_dueda_6M,m_Deuda_Var-6,m_Rel_VarDeuda_Deuda,m_Sin_Ficha,m_Malos,m_Q_irregular_12m,hora_sin,hora_cos,o_Date_2024-02-03 00:00:00,o_Date_2024-02-08 00:00:00,o_Date_2024-02-09 00:00:00,o_Date_2024-02-12 00:00:00,o_Date_2024-02-13 00:00:00,o_Date_2024-02-14 00:00:00,o_Date_2024-02-15 00:00:00,o_Date_2024-02-16 00:00:00,o_Date_2024-02-17 00:00:00,o_Date_2024-02-18 00:00:00,o_Date_2024-02-19 00:00:00,o_Date_2024-02-20 00:00:00,o_Date_2024-02-21 00:00:00,o_Date_2024-02-22 00:00:00,o_Date_2024-02-23 00:00:00,o_Date_2024-02-24 00:00:00,o_Date_2024-02-25 00:00:00,o_Date_2024-02-26 00:00:00,o_Date_2024-02-27 00:00:00,o_Date_2024-02-28 00:00:00,o_Date_2024-03-01 00:00:00,o_Date_2024-03-03 00:00:00,o_Date_2024-03-04 00:00:00,o_Date_2024-03-06 00:00:00,o_Date_2024-03-07 00:00:00,o_Date_2024-03-08 00:00:00,o_Date_2024-03-09 00:00:00,o_Date_2024-03-10 00:00:00,o_Date_2024-03-11 00:00:00,o_Date_2024-03-12 00:00:00,o_Date_2024-03-13 00:00:00,o_Date_2024-03-14 00:00:00,o_Date_2024-03-15 00:00:00,o_Date_2024-03-16 00:00:00,o_Date_2024-03-17 00:00:00,o_Date_2024-03-18 00:00:00,o_Date_2024-03-19 00:00:00,o_Date_2024-03-20 00:00:00,o_Date_2024-03-21 00:00:00,o_Date_2024-03-22 00:00:00,o_Date_2024-03-23 00:00:00,o_Date_2024-03-24 00:00:00,o_Date_2024-03-25 00:00:00,o_Date_2024-03-26 00:00:00,o_Date_2024-03-27 00:00:00,o_Date_2024-03-28 00:00:00,o_Date_2024-04-01 00:00:00,o_Date_2024-04-02 00:00:00,o_Date_2024-04-03 00:00:00,o_Date_2024-04-04 00:00:00,o_Date_2024-04-05 00:00:00,o_Date_2024-04-08 00:00:00,o_Date_2024-04-09 00:00:00,o_Date_2024-04-10 00:00:00,o_Date_2024-04-11 00:00:00,o_Date_2024-04-12 00:00:00,o_Date_2024-04-13 00:00:00,o_Date_2024-04-14 00:00:00,o_Date_2024-04-15 00:00:00,o_Date_2024-04-16 00:00:00,o_Date_2024-04-17 00:00:00,o_Date_2024-04-18 00:00:00,o_Date_2024-04-19 00:00:00,o_Date_2024-04-20 00:00:00,o_Date_2024-04-21 00:00:00,o_Date_2024-04-22 00:00:00,o_Date_2024-04-23 00:00:00,o_Date_2024-04-24 00:00:00,o_Date_2024-04-25 00:00:00,o_Date_2024-04-26 00:00:00,o_Date_2024-04-27 00:00:00,o_Date_2024-04-28 00:00:00,o_Date_2024-04-29 00:00:00,o_Date_2024-04-30 00:00:00,o_Date_2024-05-01 00:00:00,o_Date_2024-05-02 00:00:00,o_Date_2024-05-03 00:00:00,o_Date_2024-05-04 00:00:00,o_Date_2024-05-06 00:00:00,o_Date_2024-05-07 00:00:00,o_Date_2024-05-08 00:00:00,o_Date_2024-05-13 00:00:00,o_Date_2024-05-15 00:00:00,o_Date_2024-05-16 00:00:00,o_Date_2024-05-17 00:00:00,o_Date_2024-05-18 00:00:00,o_Date_2024-05-19 00:00:00,o_Date_2024-05-20 00:00:00,o_Date_2024-05-21 00:00:00,o_Date_2024-05-22 00:00:00,o_Date_2024-05-23 00:00:00,o_Date_2024-05-24 00:00:00,o_Date_2024-05-25 00:00:00,o_Date_2024-05-27 00:00:00,o_Date_2024-05-28 00:00:00,o_Date_2024-05-29 00:00:00,o_Date_2024-05-30 00:00:00,o_Date_2024-05-31 00:00:00,o_Date_2024-06-02 00:00:00,o_Date_2024-06-03 00:00:00,o_Date_2024-06-04 00:00:00,o_Date_2024-06-05 00:00:00,o_Date_2024-06-06 00:00:00,o_Date_2024-06-07 00:00:00,o_Date_2024-06-08 00:00:00,o_Date_2024-06-10 00:00:00,o_Date_2024-06-11 00:00:00,o_Date_2024-06-12 00:00:00,o_Date_2024-06-13 00:00:00,o_Date_2024-06-14 00:00:00,o_Date_2024-06-15 00:00:00,o_Date_2024-06-17 00:00:00,o_Date_2024-06-18 00:00:00,o_Date_2024-06-19 00:00:00,o_Date_2024-06-20 00:00:00,o_Date_2024-06-21 00:00:00,o_Date_2024-06-24 00:00:00,o_Date_2024-06-25 00:00:00,o_Date_2024-06-26 00:00:00,o_Date_2024-06-27 00:00:00,o_Date_2024-07-30 00:00:00,o_Date_2024-08-05 00:00:00,o_Date_2024-08-06 00:00:00,o_Date_2024-08-07 00:00:00,o_Date_2024-08-08 00:00:00,o_Date_2024-08-09 00:00:00,o_Date_2024-08-10 00:00:00,o_Date_2024-08-12 00:00:00,o_Date_2024-08-13 00:00:00,o_Date_2024-08-14 00:00:00,o_Date_2024-08-15 00:00:00,o_Date_2024-08-16 00:00:00,o_Date_2024-08-19 00:00:00,o_Date_2024-08-20 00:00:00,o_Date_2024-08-22 00:00:00,o_Da


Processed training data info:
<class 'pandas.core.frame.DataFrame'>
Index: 1277 entries, 39134533 to 35402500
Columns: 856 entries, o_quincena to m_TieneFin
dtypes: bool(12), float64(844)
memory usage: 8.2 MB

Processed test data info:
<class 'pandas.core.frame.DataFrame'>
Index: 548 entries, 47422954 to 42965420
Columns: 856 entries, o_quincena to m_TieneFin
dtypes: bool(12), float64(844)
memory usage: 3.5 MB


In [42]:
X_train_processed.shape, X_test_processed.shape

((1277, 856), (548, 856))

In [43]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_processed, y_train)

KNeighborsClassifier()

In [44]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation with 7 folds
cv_scores = cross_val_score(knn, X_train_processed, y_train, cv=7, scoring="f1")

print("Results with defaults:")
print(f"F-score: {cv_scores.mean():0.3f}±{cv_scores.std():0.3f}")

Results with defaults:
F-score: 0.552±0.025


In [45]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

# MinMax Scaling
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train_processed)
X_test_minmax = scaler_minmax.transform(X_test_processed)

# Standardization (Mean 0, Std 1)
scaler_std = StandardScaler()
X_train_std = scaler_std.fit_transform(X_train_processed)
X_test_std = scaler_std.transform(X_test_processed)

# Train KNN with default parameters using MinMax Scaled data
knn_minmax = KNeighborsClassifier()
cv_scores_minmax = cross_val_score(knn_minmax, X_train_minmax, y_train, cv=7, scoring="f1")

print("Results with MinMax Scaling:")
print(f"F-score: {cv_scores_minmax.mean():0.3f}±{cv_scores_minmax.std():0.3f}")
print("-" * 20)

# Train KNN with default parameters using Standardized data
knn_std = KNeighborsClassifier()
cv_scores_std = cross_val_score(knn_std, X_train_std, y_train, cv=7, scoring="f1")

print("Results with Standardization:")
print(f"F-score: {cv_scores_std.mean():0.3f}±{cv_scores_std.std():0.3f}")

Results with MinMax Scaling:
F-score: 0.552±0.025
--------------------
Results with Standardization:
F-score: 0.423±0.043


In [46]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Usamos f_classif que mide la dependencia lineal entre features y el target
# Aunque no es correlación exacta, es una medida de la relación lineal.
# Seleccionamos las 14 mejores características
selector = SelectKBest(score_func=f_classif, k=14)

# Ajustamos el selector a los datos de entrenamiento y el target
selector.fit(X_train_processed, y_train)

# Obtenemos las columnas seleccionadas (indices booleanos)
selected_features_mask = selector.get_support()

# Obtenemos los nombres de las columnas seleccionadas
selected_feature_names = X_train_processed.columns[selected_features_mask]

# Transformamos los datos de entrenamiento y prueba para quedarnos solo con las features seleccionadas
X_train_selected_k = selector.transform(X_train_processed)
X_test_selected_k = selector.transform(X_test_processed)

print("Features seleccionadas:")
print(selected_feature_names)
print("\nDimensiones de los datos de entrenamiento después de la selección:", X_train_selected_k.shape) # Corrected variable name
print("Dimensiones de los datos de prueba después de la selección:", X_test_selected_k.shape)       # Corrected variable name

Features seleccionadas:
Index(['o_edad', 'o_clr_Score3', 'o_cdr_segmento', 'o_pricing', 'm_Entidades',
       'm_Var_dueda_6M', 'm_Rel_VarDeuda_Deuda', 'm_Sin_Ficha', 'm_Grupo_b_C',
       'm_Grupo_b_R', 'm_Grupo_i_A', 'm_Grupo_i_C', 'o_clr_Incumplimientos',
       'm_TieneFin'],
      dtype='object')

Dimensiones de los datos de entrenamiento después de la selección: (1277, 14)
Dimensiones de los datos de prueba después de la selección: (548, 14)


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [844 845 846 847 848 851] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [47]:
from sklearn.metrics import f1_score

# Predict on the test data
y_pred = knn.predict(X_test_processed)

# Calculate the F1-score on the test data
test_f1 = f1_score(y_test, y_pred)

print(f"F1-score on the test set: {test_f1:.3f}")

F1-score on the test set: 0.524
